In [1]:
from functools import partial
import jax 
import jax.numpy as jnp

from jax.scipy.linalg import solve

jax.config.update("jax_enable_x64", True)



@partial(jax.jit, static_argnames=('model',))
def metric(coords, model=None):
    """
    Compute the metric tensor at a given coordinate.

    Parameters
    ----------
    coords : jnp.array
        Coordinates of the point at which to compute the metric.
    model :
        Function that returns the metric tensor at a given coordinate. If not specified, the Euclidean metric is used.
    
    Returns
    -------
    metric : jnp.array
        Metric tensor (two lower indices) at the given coordinate. 
    """

    if model is None:
        return jnp.eye(coords.shape[-1])
    return model(coords)

# partial derivatives of the metric
pd_metric = jax.jit(jax.jacfwd(metric), static_argnames=('model',))

@partial(jax.jit, static_argnames=('model',))
def christoffel(coords, model):
    """
    Compute the Christoffel symbols at a given coordinate.

    Parameters
    ----------
    coords : jnp.array
        Coordinates of the point at which to compute the Christoffel symbols.
    model :
        Function that returns the metric at a given coordinate.
    
    Returns
    -------
    christoffel : jnp.array
        Christoffel symbols at the given coordinate. The first index is upper, the rest two indices are lower.
    """

    met = metric(coords, model)
    pd = jnp.einsum('mns -> smn', pd_metric(coords, model))
    rhs = pd + jnp.einsum('nrm -> mnr', pd) - jnp.einsum('rmn -> mnr', pd)
    rhs = 0.5 * rhs  # shape (dim, dim, dim)
    rhs =  jnp.einsum('rmn -> nrm', rhs)
    # Solve g_{rs} X^s_{mn} = rhs_{r mn}
    # reshape to solve all RHS at once
    dim = met.shape[0]
    rhs_flat = rhs.reshape(dim, -1)

    sol_flat = solve(met, rhs_flat, assume_a='gen')
    christ = sol_flat.reshape(dim, dim, dim)

    return christ

# partial derivatives of the christoffel symbols
pd_christoffel = jax.jit(jax.jacfwd(christoffel), static_argnames=('model',))

@partial(jax.jit, static_argnames=('model',))
def riemann_curvature(coords, model):
    """
    Compute the Riemann curvature at a given coordinate.

    Parameters
    ----------
    coords : jnp.array
        Coordinates of the point at which to compute the Riemann curvature.
    model :
        Function that returns the metric at a given coordinate.
    
    Returns
    -------
    riemann_curvature : jnp.array
        Riemann curvature at the given coordinate. The first index is upper, the rest three indices are lower.
    """

    christ = christoffel(coords, model)
    pd_christ = jnp.einsum('rmns -> srmn', pd_christoffel(coords, model))
    return jnp.einsum('mrns -> rsmn', pd_christ) - jnp.einsum('nrms -> rsmn', pd_christ) + jnp.einsum('rml, lns -> rsmn', christ, christ) - jnp.einsum('rnl, lms -> rsmn', christ, christ)

@partial(jax.jit, static_argnames=('model',))
def ricci_tensor(coords, model):
    """
    Compute the Ricci tensor at a given coordinate.

    Parameters
    ----------
    coords : jnp.array
        Coordinates of the point at which to compute the Ricci tensor.
    model :
        Function that returns the metric at a given coordinate.
    
    Returns
    -------
    ricci_tensor : jnp.array
        Ricci tensor (two lower indices) at the given coordinate. 
    """

    riemann = riemann_curvature(coords, model)
    return jnp.einsum('rsru -> su', riemann)

@partial(jax.jit, static_argnames=('model',))
def ricci_scalar(coords, model):
    """
    Compute the Ricci scalar at a given coordinate.

    Parameters
    ----------
    coords : jnp.array
        Coordinates of the point at which to compute the Ricci scalar.
    model :
        Function that returns the metric at a given coordinate.
    
    Returns
    -------
    ricci_scalar : jnp.array
        Ricci scalar at the given coordinate. 
    """

    return jnp.einsum('mn, mn -> ', jnp.linalg.inv(metric(coords, model)), ricci_tensor(coords, model))

@partial(jax.jit, static_argnames=('model',))
def einstein_tensor(coords, model):
    """
    Compute the Einstein tensor at a given coordinate.

    Parameters
    ----------
    coords : jnp.array
        Coordinates of the point at which to compute the Einstein tensor.
    model :
        Function that returns the metric at a given coordinate.
    
    Returns
    -------
    einstein_tensor : jnp.array
        Einstein tensor (two lower indices) at the given coordinate. 
    """
    met = metric(coords, model)
    ricci_ts = ricci_tensor(coords, model)
    return ricci_ts - 0.5 * jnp.einsum('mn, mn -> ', jnp.linalg.inv(met), ricci_ts).reshape(1, 1) * met


In [2]:
def Kerr_metric(a,Q,rs):
    m=rs/2
    @jax.jit
    def Kerr(coords):
        t, x, y, z = coords[0],coords[1],coords[2],coords[3]
        r=x
        delta=r**2-2*m*r+a**2+Q**2
        rho2=r**2+(a**2)*(jnp.cos(y)**2)
        return jnp.array([
            [-a**2*jnp.sin(y)**2/rho2+delta/rho2,0,0,(a**3*jnp.sin(y)**2+a*r**2*jnp.sin(y)**2-delta*a*jnp.sin(y)**2)/rho2],
            [0,-rho2/delta,0,0],
            [0,0,-rho2,0],
            [(a**3*jnp.sin(y)**2+a*r**2*jnp.sin(y)**2-delta*a*jnp.sin(y)**2)/rho2,0,0,(delta*a**2*jnp.sin(y)**4-a**4*jnp.sin(y)**2-2*a**2*r**2*jnp.sin(y)**2-r**4*jnp.sin(y)**2)/rho2]
        ])
    return Kerr


In [3]:
import jax.numpy as math
@jax.jit
def ch(coords,a,Q,rs):
    t, x, y, z = coords[0],coords[1],coords[2],coords[3]
    return jnp.array([[[0, (Q**2*a*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - a*rs*x**2*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + (1/2)*a*rs*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2))*(-Q**2*a**3*math.cos(y)**2 - Q**2*a*x**2 + a**3*rs*x*math.cos(y)**2 + a*rs*x**3)/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6) + (-Q**2*x/(a**2*math.cos(y)**2 + x**2)**2 + a**2*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - a**2*x/(a**2*math.cos(y)**2 + x**2)**2 + rs*x**2/(a**2*math.cos(y)**2 + x**2)**2 - 1/2*rs/(a**2*math.cos(y)**2 + x**2) - x**3/(a**2*math.cos(y)**2 + x**2)**2 + x/(a**2*math.cos(y)**2 + x**2))*(-Q**2*a**4*math.sin(y)**2*math.cos(y)**2 - Q**2*a**2*x**2*math.sin(y)**2 - a**6*math.sin(y)**2*math.cos(y)**2 + a**6*math.cos(y)**2 + a**4*rs*x*math.sin(y)**2*math.cos(y)**2 - a**4*x**2*math.sin(y)**2*math.cos(y)**2 - a**4*x**2*math.sin(y)**2 + 2*a**4*x**2*math.cos(y)**2 + a**4*x**2 + a**2*rs*x**3*math.sin(y)**2 - a**2*x**4*math.sin(y)**2 + a**2*x**4*math.cos(y)**2 + 2*a**2*x**4 + x**6)/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6), (-Q**2*a**3*math.cos(y)**2 - Q**2*a*x**2 + a**3*rs*x*math.cos(y)**2 + a*rs*x**3)*(-Q**2*a**3*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - Q**2*a*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + a**3*rs*x*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a*rs*x*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6) + (Q**2*a**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**4*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**4*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**2*rs*x*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**2*x**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))*(-Q**2*a**4*math.sin(y)**2*math.cos(y)**2 - Q**2*a**2*x**2*math.sin(y)**2 - a**6*math.sin(y)**2*math.cos(y)**2 + a**6*math.cos(y)**2 + a**4*rs*x*math.sin(y)**2*math.cos(y)**2 - a**4*x**2*math.sin(y)**2*math.cos(y)**2 - a**4*x**2*math.sin(y)**2 + 2*a**4*x**2*math.cos(y)**2 + a**4*x**2 + a**2*rs*x**3*math.sin(y)**2 - a**2*x**4*math.sin(y)**2 + a**2*x**4*math.cos(y)**2 + 2*a**2*x**4 + x**6)/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6), 0], [(Q**2*a*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - a*rs*x**2*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + (1/2)*a*rs*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2))*(-Q**2*a**3*math.cos(y)**2 - Q**2*a*x**2 + a**3*rs*x*math.cos(y)**2 + a*rs*x**3)/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6) + (-Q**2*x/(a**2*math.cos(y)**2 + x**2)**2 + a**2*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - a**2*x/(a**2*math.cos(y)**2 + x**2)**2 + rs*x**2/(a**2*math.cos(y)**2 + x**2)**2 - 1/2*rs/(a**2*math.cos(y)**2 + x**2) - x**3/(a**2*math.cos(y)**2 + x**2)**2 + x/(a**2*math.cos(y)**2 + x**2))*(-Q**2*a**4*math.sin(y)**2*math.cos(y)**2 - Q**2*a**2*x**2*math.sin(y)**2 - a**6*math.sin(y)**2*math.cos(y)**2 + a**6*math.cos(y)**2 + a**4*rs*x*math.sin(y)**2*math.cos(y)**2 - a**4*x**2*math.sin(y)**2*math.cos(y)**2 - a**4*x**2*math.sin(y)**2 + 2*a**4*x**2*math.cos(y)**2 + a**4*x**2 + a**2*rs*x**3*math.sin(y)**2 - a**2*x**4*math.sin(y)**2 + a**2*x**4*math.cos(y)**2 + 2*a**2*x**4 + x**6)/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6), 0, 0, (Q**2*a*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - a*rs*x**2*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + (1/2)*a*rs*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2))*(-Q**2*a**4*math.sin(y)**2*math.cos(y)**2 - Q**2*a**2*x**2*math.sin(y)**2 - a**6*math.sin(y)**2*math.cos(y)**2 + a**6*math.cos(y)**2 + a**4*rs*x*math.sin(y)**2*math.cos(y)**2 - a**4*x**2*math.sin(y)**2*math.cos(y)**2 - a**4*x**2*math.sin(y)**2 + 2*a**4*x**2*math.cos(y)**2 + a**4*x**2 + a**2*rs*x**3*math.sin(y)**2 - a**2*x**4*math.sin(y)**2 + a**2*x**4*math.cos(y)**2 + 2*a**2*x**4 + x**6)/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6) + (-Q**2*a**3*math.cos(y)**2 - Q**2*a*x**2 + a**3*rs*x*math.cos(y)**2 + a*rs*x**3)*(-Q**2*a**2*x*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 - a**4*x*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 + a**4*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + a**2*rs*x**2*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 - 1/2*a**2*rs*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2) - a**2*x**3*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 + 2*a**2*x**3*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + a**2*x*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2) - 2*a**2*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2) + x**5*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - 2*x**3*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2))/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6)], [(-Q**2*a**3*math.cos(y)**2 - Q**2*a*x**2 + a**3*rs*x*math.cos(y)**2 + a*rs*x**3)*(-Q**2*a**3*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - Q**2*a*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + a**3*rs*x*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a*rs*x*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6) + (Q**2*a**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**4*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**4*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**2*rs*x*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**2*x**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))*(-Q**2*a**4*math.sin(y)**2*math.cos(y)**2 - Q**2*a**2*x**2*math.sin(y)**2 - a**6*math.sin(y)**2*math.cos(y)**2 + a**6*math.cos(y)**2 + a**4*rs*x*math.sin(y)**2*math.cos(y)**2 - a**4*x**2*math.sin(y)**2*math.cos(y)**2 - a**4*x**2*math.sin(y)**2 + 2*a**4*x**2*math.cos(y)**2 + a**4*x**2 + a**2*rs*x**3*math.sin(y)**2 - a**2*x**4*math.sin(y)**2 + a**2*x**4*math.cos(y)**2 + 2*a**2*x**4 + x**6)/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6), 0, 0, (-Q**2*a**3*math.cos(y)**2 - Q**2*a*x**2 + a**3*rs*x*math.cos(y)**2 + a*rs*x**3)*(Q**2*a**4*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + 2*Q**2*a**2*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + a**6*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**6*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**4*rs*x*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**4*x**2*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - 2*a**4*x**2*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + 2*a**4*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - a**4*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - 2*a**2*rs*x*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - a**2*x**4*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + 2*a**2*x**2*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - 2*a**2*x**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - x**4*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6) + (-Q**2*a**3*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - Q**2*a*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + a**3*rs*x*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a*rs*x*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))*(-Q**2*a**4*math.sin(y)**2*math.cos(y)**2 - Q**2*a**2*x**2*math.sin(y)**2 - a**6*math.sin(y)**2*math.cos(y)**2 + a**6*math.cos(y)**2 + a**4*rs*x*math.sin(y)**2*math.cos(y)**2 - a**4*x**2*math.sin(y)**2*math.cos(y)**2 - a**4*x**2*math.sin(y)**2 + 2*a**4*x**2*math.cos(y)**2 + a**4*x**2 + a**2*rs*x**3*math.sin(y)**2 - a**2*x**4*math.sin(y)**2 + a**2*x**4*math.cos(y)**2 + 2*a**2*x**4 + x**6)/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6)], [0, (Q**2*a*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - a*rs*x**2*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + (1/2)*a*rs*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2))*(-Q**2*a**4*math.sin(y)**2*math.cos(y)**2 - Q**2*a**2*x**2*math.sin(y)**2 - a**6*math.sin(y)**2*math.cos(y)**2 + a**6*math.cos(y)**2 + a**4*rs*x*math.sin(y)**2*math.cos(y)**2 - a**4*x**2*math.sin(y)**2*math.cos(y)**2 - a**4*x**2*math.sin(y)**2 + 2*a**4*x**2*math.cos(y)**2 + a**4*x**2 + a**2*rs*x**3*math.sin(y)**2 - a**2*x**4*math.sin(y)**2 + a**2*x**4*math.cos(y)**2 + 2*a**2*x**4 + x**6)/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6) + (-Q**2*a**3*math.cos(y)**2 - Q**2*a*x**2 + a**3*rs*x*math.cos(y)**2 + a*rs*x**3)*(-Q**2*a**2*x*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 - a**4*x*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 + a**4*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + a**2*rs*x**2*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 - 1/2*a**2*rs*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2) - a**2*x**3*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 + 2*a**2*x**3*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + a**2*x*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2) - 2*a**2*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2) + x**5*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - 2*x**3*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2))/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6), (-Q**2*a**3*math.cos(y)**2 - Q**2*a*x**2 + a**3*rs*x*math.cos(y)**2 + a*rs*x**3)*(Q**2*a**4*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + 2*Q**2*a**2*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + a**6*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**6*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**4*rs*x*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**4*x**2*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - 2*a**4*x**2*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + 2*a**4*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - a**4*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - 2*a**2*rs*x*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - a**2*x**4*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + 2*a**2*x**2*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - 2*a**2*x**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - x**4*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6) + (-Q**2*a**3*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - Q**2*a*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + a**3*rs*x*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a*rs*x*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))*(-Q**2*a**4*math.sin(y)**2*math.cos(y)**2 - Q**2*a**2*x**2*math.sin(y)**2 - a**6*math.sin(y)**2*math.cos(y)**2 + a**6*math.cos(y)**2 + a**4*rs*x*math.sin(y)**2*math.cos(y)**2 - a**4*x**2*math.sin(y)**2*math.cos(y)**2 - a**4*x**2*math.sin(y)**2 + 2*a**4*x**2*math.cos(y)**2 + a**4*x**2 + a**2*rs*x**3*math.sin(y)**2 - a**2*x**4*math.sin(y)**2 + a**2*x**4*math.cos(y)**2 + 2*a**2*x**4 + x**6)/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6), 0]], [[(-Q**2 - a**2 + rs*x - x**2)*(Q**2*x/(a**2*math.cos(y)**2 + x**2)**2 - a**2*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + a**2*x/(a**2*math.cos(y)**2 + x**2)**2 - rs*x**2/(a**2*math.cos(y)**2 + x**2)**2 + (1/2)*rs/(a**2*math.cos(y)**2 + x**2) + x**3/(a**2*math.cos(y)**2 + x**2)**2 - x/(a**2*math.cos(y)**2 + x**2))/(a**2*math.cos(y)**2 + x**2), 0, 0, (-Q**2*a*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + a*rs*x**2*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - 1/2*a*rs*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2))*(-Q**2 - a**2 + rs*x - x**2)/(a**2*math.cos(y)**2 + x**2)], [0, (-1/2*a**2*(rs - 2*x)*math.cos(y)**2/(Q**2 + a**2 - rs*x + x**2)**2 - 1/2*x**2*(rs - 2*x)/(Q**2 + a**2 - rs*x + x**2)**2 - x/(Q**2 + a**2 - rs*x + x**2))*(-Q**2 - a**2 + rs*x - x**2)/(a**2*math.cos(y)**2 + x**2), a**2*(-Q**2 - a**2 + rs*x - x**2)*math.sin(y)*math.cos(y)/((a**2*math.cos(y)**2 + x**2)*(Q**2 + a**2 - rs*x + x**2)), 0], [0, a**2*(-Q**2 - a**2 + rs*x - x**2)*math.sin(y)*math.cos(y)/((a**2*math.cos(y)**2 + x**2)*(Q**2 + a**2 - rs*x + x**2)), x*(-Q**2 - a**2 + rs*x - x**2)/(a**2*math.cos(y)**2 + x**2), 0], [(-Q**2*a*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + a*rs*x**2*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - 1/2*a*rs*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2))*(-Q**2 - a**2 + rs*x - x**2)/(a**2*math.cos(y)**2 + x**2), 0, 0, (-Q**2 - a**2 + rs*x - x**2)*(Q**2*a**2*x*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 + a**4*x*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 - a**4*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - a**2*rs*x**2*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 + (1/2)*a**2*rs*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2) + a**2*x**3*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 - 2*a**2*x**3*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - a**2*x*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2) + 2*a**2*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2) - x**5*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + 2*x**3*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2))/(a**2*math.cos(y)**2 + x**2)]], [[-(-Q**2*a**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**4*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**4*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**2*rs*x*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**2*x**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))/(a**2*math.cos(y)**2 + x**2), 0, 0, -(Q**2*a**3*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + Q**2*a*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - a**3*rs*x*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a*rs*x*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))/(a**2*math.cos(y)**2 + x**2)], [0, a**2*math.sin(y)*math.cos(y)/((a**2*math.cos(y)**2 + x**2)*(Q**2 + a**2 - rs*x + x**2)), x/(a**2*math.cos(y)**2 + x**2), 0], [0, x/(a**2*math.cos(y)**2 + x**2), -a**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2), 0], [-(Q**2*a**3*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + Q**2*a*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - a**3*rs*x*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a*rs*x*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))/(a**2*math.cos(y)**2 + x**2), 0, 0, -(-Q**2*a**4*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - 2*Q**2*a**2*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - a**6*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**6*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**4*rs*x*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**4*x**2*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + 2*a**4*x**2*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - 2*a**4*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + a**4*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + 2*a**2*rs*x*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + a**2*x**4*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - 2*a**2*x**2*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + 2*a**2*x**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + x**4*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))/(a**2*math.cos(y)**2 + x**2)]], [[0, (Q**2*a*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - a*rs*x**2*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + (1/2)*a*rs*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2))*(-Q**2*a**2*math.cos(y)**2 - Q**2*x**2 + a**4*math.sin(y)**2*math.cos(y)**2 - a**4*math.cos(y)**2 + a**2*rs*x*math.cos(y)**2 + a**2*x**2*math.sin(y)**2 - a**2*x**2*math.cos(y)**2 - a**2*x**2 + rs*x**3 - x**4)/(Q**2*a**4*math.sin(y)**6 - 2*Q**2*a**4*math.sin(y)**4 + Q**2*a**4*math.sin(y)**2 - 2*Q**2*a**2*x**2*math.sin(y)**4 + 2*Q**2*a**2*x**2*math.sin(y)**2 + Q**2*x**4*math.sin(y)**2 + a**6*math.sin(y)**6 - 2*a**6*math.sin(y)**4 + a**6*math.sin(y)**2 - a**4*rs*x*math.sin(y)**6 + 2*a**4*rs*x*math.sin(y)**4 - a**4*rs*x*math.sin(y)**2 + a**4*x**2*math.sin(y)**6 - 4*a**4*x**2*math.sin(y)**4 + 3*a**4*x**2*math.sin(y)**2 + 2*a**2*rs*x**3*math.sin(y)**4 - 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*x**4*math.sin(y)**4 + 3*a**2*x**4*math.sin(y)**2 - rs*x**5*math.sin(y)**2 + x**6*math.sin(y)**2) + (-Q**2*a**3*math.cos(y)**2 - Q**2*a*x**2 + a**3*rs*x*math.cos(y)**2 + a*rs*x**3)*(-Q**2*x/(a**2*math.cos(y)**2 + x**2)**2 + a**2*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - a**2*x/(a**2*math.cos(y)**2 + x**2)**2 + rs*x**2/(a**2*math.cos(y)**2 + x**2)**2 - 1/2*rs/(a**2*math.cos(y)**2 + x**2) - x**3/(a**2*math.cos(y)**2 + x**2)**2 + x/(a**2*math.cos(y)**2 + x**2))/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6), (-Q**2*a**3*math.cos(y)**2 - Q**2*a*x**2 + a**3*rs*x*math.cos(y)**2 + a*rs*x**3)*(Q**2*a**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**4*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**4*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**2*rs*x*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**2*x**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6) + (-Q**2*a**3*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - Q**2*a*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + a**3*rs*x*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a*rs*x*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))*(-Q**2*a**2*math.cos(y)**2 - Q**2*x**2 + a**4*math.sin(y)**2*math.cos(y)**2 - a**4*math.cos(y)**2 + a**2*rs*x*math.cos(y)**2 + a**2*x**2*math.sin(y)**2 - a**2*x**2*math.cos(y)**2 - a**2*x**2 + rs*x**3 - x**4)/(Q**2*a**4*math.sin(y)**6 - 2*Q**2*a**4*math.sin(y)**4 + Q**2*a**4*math.sin(y)**2 - 2*Q**2*a**2*x**2*math.sin(y)**4 + 2*Q**2*a**2*x**2*math.sin(y)**2 + Q**2*x**4*math.sin(y)**2 + a**6*math.sin(y)**6 - 2*a**6*math.sin(y)**4 + a**6*math.sin(y)**2 - a**4*rs*x*math.sin(y)**6 + 2*a**4*rs*x*math.sin(y)**4 - a**4*rs*x*math.sin(y)**2 + a**4*x**2*math.sin(y)**6 - 4*a**4*x**2*math.sin(y)**4 + 3*a**4*x**2*math.sin(y)**2 + 2*a**2*rs*x**3*math.sin(y)**4 - 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*x**4*math.sin(y)**4 + 3*a**2*x**4*math.sin(y)**2 - rs*x**5*math.sin(y)**2 + x**6*math.sin(y)**2), 0], [(Q**2*a*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - a*rs*x**2*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + (1/2)*a*rs*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2))*(-Q**2*a**2*math.cos(y)**2 - Q**2*x**2 + a**4*math.sin(y)**2*math.cos(y)**2 - a**4*math.cos(y)**2 + a**2*rs*x*math.cos(y)**2 + a**2*x**2*math.sin(y)**2 - a**2*x**2*math.cos(y)**2 - a**2*x**2 + rs*x**3 - x**4)/(Q**2*a**4*math.sin(y)**6 - 2*Q**2*a**4*math.sin(y)**4 + Q**2*a**4*math.sin(y)**2 - 2*Q**2*a**2*x**2*math.sin(y)**4 + 2*Q**2*a**2*x**2*math.sin(y)**2 + Q**2*x**4*math.sin(y)**2 + a**6*math.sin(y)**6 - 2*a**6*math.sin(y)**4 + a**6*math.sin(y)**2 - a**4*rs*x*math.sin(y)**6 + 2*a**4*rs*x*math.sin(y)**4 - a**4*rs*x*math.sin(y)**2 + a**4*x**2*math.sin(y)**6 - 4*a**4*x**2*math.sin(y)**4 + 3*a**4*x**2*math.sin(y)**2 + 2*a**2*rs*x**3*math.sin(y)**4 - 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*x**4*math.sin(y)**4 + 3*a**2*x**4*math.sin(y)**2 - rs*x**5*math.sin(y)**2 + x**6*math.sin(y)**2) + (-Q**2*a**3*math.cos(y)**2 - Q**2*a*x**2 + a**3*rs*x*math.cos(y)**2 + a*rs*x**3)*(-Q**2*x/(a**2*math.cos(y)**2 + x**2)**2 + a**2*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - a**2*x/(a**2*math.cos(y)**2 + x**2)**2 + rs*x**2/(a**2*math.cos(y)**2 + x**2)**2 - 1/2*rs/(a**2*math.cos(y)**2 + x**2) - x**3/(a**2*math.cos(y)**2 + x**2)**2 + x/(a**2*math.cos(y)**2 + x**2))/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6), 0, 0, (Q**2*a*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - a*rs*x**2*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + (1/2)*a*rs*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2))*(-Q**2*a**3*math.cos(y)**2 - Q**2*a*x**2 + a**3*rs*x*math.cos(y)**2 + a*rs*x**3)/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6) + (-Q**2*a**2*math.cos(y)**2 - Q**2*x**2 + a**4*math.sin(y)**2*math.cos(y)**2 - a**4*math.cos(y)**2 + a**2*rs*x*math.cos(y)**2 + a**2*x**2*math.sin(y)**2 - a**2*x**2*math.cos(y)**2 - a**2*x**2 + rs*x**3 - x**4)*(-Q**2*a**2*x*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 - a**4*x*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 + a**4*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + a**2*rs*x**2*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 - 1/2*a**2*rs*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2) - a**2*x**3*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 + 2*a**2*x**3*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + a**2*x*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2) - 2*a**2*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2) + x**5*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - 2*x**3*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2))/(Q**2*a**4*math.sin(y)**6 - 2*Q**2*a**4*math.sin(y)**4 + Q**2*a**4*math.sin(y)**2 - 2*Q**2*a**2*x**2*math.sin(y)**4 + 2*Q**2*a**2*x**2*math.sin(y)**2 + Q**2*x**4*math.sin(y)**2 + a**6*math.sin(y)**6 - 2*a**6*math.sin(y)**4 + a**6*math.sin(y)**2 - a**4*rs*x*math.sin(y)**6 + 2*a**4*rs*x*math.sin(y)**4 - a**4*rs*x*math.sin(y)**2 + a**4*x**2*math.sin(y)**6 - 4*a**4*x**2*math.sin(y)**4 + 3*a**4*x**2*math.sin(y)**2 + 2*a**2*rs*x**3*math.sin(y)**4 - 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*x**4*math.sin(y)**4 + 3*a**2*x**4*math.sin(y)**2 - rs*x**5*math.sin(y)**2 + x**6*math.sin(y)**2)], [(-Q**2*a**3*math.cos(y)**2 - Q**2*a*x**2 + a**3*rs*x*math.cos(y)**2 + a*rs*x**3)*(Q**2*a**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**4*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**4*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**2*rs*x*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**2*x**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6) + (-Q**2*a**3*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - Q**2*a*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + a**3*rs*x*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a*rs*x*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))*(-Q**2*a**2*math.cos(y)**2 - Q**2*x**2 + a**4*math.sin(y)**2*math.cos(y)**2 - a**4*math.cos(y)**2 + a**2*rs*x*math.cos(y)**2 + a**2*x**2*math.sin(y)**2 - a**2*x**2*math.cos(y)**2 - a**2*x**2 + rs*x**3 - x**4)/(Q**2*a**4*math.sin(y)**6 - 2*Q**2*a**4*math.sin(y)**4 + Q**2*a**4*math.sin(y)**2 - 2*Q**2*a**2*x**2*math.sin(y)**4 + 2*Q**2*a**2*x**2*math.sin(y)**2 + Q**2*x**4*math.sin(y)**2 + a**6*math.sin(y)**6 - 2*a**6*math.sin(y)**4 + a**6*math.sin(y)**2 - a**4*rs*x*math.sin(y)**6 + 2*a**4*rs*x*math.sin(y)**4 - a**4*rs*x*math.sin(y)**2 + a**4*x**2*math.sin(y)**6 - 4*a**4*x**2*math.sin(y)**4 + 3*a**4*x**2*math.sin(y)**2 + 2*a**2*rs*x**3*math.sin(y)**4 - 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*x**4*math.sin(y)**4 + 3*a**2*x**4*math.sin(y)**2 - rs*x**5*math.sin(y)**2 + x**6*math.sin(y)**2), 0, 0, (-Q**2*a**3*math.cos(y)**2 - Q**2*a*x**2 + a**3*rs*x*math.cos(y)**2 + a*rs*x**3)*(-Q**2*a**3*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - Q**2*a*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + a**3*rs*x*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a*rs*x*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6) + (-Q**2*a**2*math.cos(y)**2 - Q**2*x**2 + a**4*math.sin(y)**2*math.cos(y)**2 - a**4*math.cos(y)**2 + a**2*rs*x*math.cos(y)**2 + a**2*x**2*math.sin(y)**2 - a**2*x**2*math.cos(y)**2 - a**2*x**2 + rs*x**3 - x**4)*(Q**2*a**4*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + 2*Q**2*a**2*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + a**6*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**6*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**4*rs*x*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**4*x**2*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - 2*a**4*x**2*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + 2*a**4*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - a**4*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - 2*a**2*rs*x*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - a**2*x**4*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + 2*a**2*x**2*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - 2*a**2*x**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - x**4*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))/(Q**2*a**4*math.sin(y)**6 - 2*Q**2*a**4*math.sin(y)**4 + Q**2*a**4*math.sin(y)**2 - 2*Q**2*a**2*x**2*math.sin(y)**4 + 2*Q**2*a**2*x**2*math.sin(y)**2 + Q**2*x**4*math.sin(y)**2 + a**6*math.sin(y)**6 - 2*a**6*math.sin(y)**4 + a**6*math.sin(y)**2 - a**4*rs*x*math.sin(y)**6 + 2*a**4*rs*x*math.sin(y)**4 - a**4*rs*x*math.sin(y)**2 + a**4*x**2*math.sin(y)**6 - 4*a**4*x**2*math.sin(y)**4 + 3*a**4*x**2*math.sin(y)**2 + 2*a**2*rs*x**3*math.sin(y)**4 - 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*x**4*math.sin(y)**4 + 3*a**2*x**4*math.sin(y)**2 - rs*x**5*math.sin(y)**2 + x**6*math.sin(y)**2)], [0, (Q**2*a*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - a*rs*x**2*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + (1/2)*a*rs*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2))*(-Q**2*a**3*math.cos(y)**2 - Q**2*a*x**2 + a**3*rs*x*math.cos(y)**2 + a*rs*x**3)/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6) + (-Q**2*a**2*math.cos(y)**2 - Q**2*x**2 + a**4*math.sin(y)**2*math.cos(y)**2 - a**4*math.cos(y)**2 + a**2*rs*x*math.cos(y)**2 + a**2*x**2*math.sin(y)**2 - a**2*x**2*math.cos(y)**2 - a**2*x**2 + rs*x**3 - x**4)*(-Q**2*a**2*x*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 - a**4*x*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 + a**4*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + a**2*rs*x**2*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 - 1/2*a**2*rs*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2) - a**2*x**3*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2)**2 + 2*a**2*x**3*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 + a**2*x*math.sin(y)**4/(a**2*math.cos(y)**2 + x**2) - 2*a**2*x*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2) + x**5*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2)**2 - 2*x**3*math.sin(y)**2/(a**2*math.cos(y)**2 + x**2))/(Q**2*a**4*math.sin(y)**6 - 2*Q**2*a**4*math.sin(y)**4 + Q**2*a**4*math.sin(y)**2 - 2*Q**2*a**2*x**2*math.sin(y)**4 + 2*Q**2*a**2*x**2*math.sin(y)**2 + Q**2*x**4*math.sin(y)**2 + a**6*math.sin(y)**6 - 2*a**6*math.sin(y)**4 + a**6*math.sin(y)**2 - a**4*rs*x*math.sin(y)**6 + 2*a**4*rs*x*math.sin(y)**4 - a**4*rs*x*math.sin(y)**2 + a**4*x**2*math.sin(y)**6 - 4*a**4*x**2*math.sin(y)**4 + 3*a**4*x**2*math.sin(y)**2 + 2*a**2*rs*x**3*math.sin(y)**4 - 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*x**4*math.sin(y)**4 + 3*a**2*x**4*math.sin(y)**2 - rs*x**5*math.sin(y)**2 + x**6*math.sin(y)**2), (-Q**2*a**3*math.cos(y)**2 - Q**2*a*x**2 + a**3*rs*x*math.cos(y)**2 + a*rs*x**3)*(-Q**2*a**3*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - Q**2*a*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + a**3*rs*x*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a*rs*x*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))/(Q**2*a**4*math.sin(y)**4 - 2*Q**2*a**4*math.sin(y)**2 + Q**2*a**4 - 2*Q**2*a**2*x**2*math.sin(y)**2 + 2*Q**2*a**2*x**2 + Q**2*x**4 + a**6*math.sin(y)**4 - 2*a**6*math.sin(y)**2 + a**6 - a**4*rs*x*math.sin(y)**4 + 2*a**4*rs*x*math.sin(y)**2 - a**4*rs*x + a**4*x**2*math.sin(y)**4 - 4*a**4*x**2*math.sin(y)**2 + 3*a**4*x**2 + 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*rs*x**3 - 2*a**2*x**4*math.sin(y)**2 + 3*a**2*x**4 - rs*x**5 + x**6) + (-Q**2*a**2*math.cos(y)**2 - Q**2*x**2 + a**4*math.sin(y)**2*math.cos(y)**2 - a**4*math.cos(y)**2 + a**2*rs*x*math.cos(y)**2 + a**2*x**2*math.sin(y)**2 - a**2*x**2*math.cos(y)**2 - a**2*x**2 + rs*x**3 - x**4)*(Q**2*a**4*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + 2*Q**2*a**2*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) + a**6*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**6*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - a**4*rs*x*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + a**4*x**2*math.sin(y)**5*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 - 2*a**4*x**2*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + 2*a**4*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - a**4*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - 2*a**2*rs*x*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - a**2*x**4*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2)**2 + 2*a**2*x**2*math.sin(y)**3*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - 2*a**2*x**2*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2) - x**4*math.sin(y)*math.cos(y)/(a**2*math.cos(y)**2 + x**2))/(Q**2*a**4*math.sin(y)**6 - 2*Q**2*a**4*math.sin(y)**4 + Q**2*a**4*math.sin(y)**2 - 2*Q**2*a**2*x**2*math.sin(y)**4 + 2*Q**2*a**2*x**2*math.sin(y)**2 + Q**2*x**4*math.sin(y)**2 + a**6*math.sin(y)**6 - 2*a**6*math.sin(y)**4 + a**6*math.sin(y)**2 - a**4*rs*x*math.sin(y)**6 + 2*a**4*rs*x*math.sin(y)**4 - a**4*rs*x*math.sin(y)**2 + a**4*x**2*math.sin(y)**6 - 4*a**4*x**2*math.sin(y)**4 + 3*a**4*x**2*math.sin(y)**2 + 2*a**2*rs*x**3*math.sin(y)**4 - 2*a**2*rs*x**3*math.sin(y)**2 - 2*a**2*x**4*math.sin(y)**4 + 3*a**2*x**4*math.sin(y)**2 - rs*x**5*math.sin(y)**2 + x**6*math.sin(y)**2), 0]]])

In [15]:
coord=jnp.array([0.2,1.5,1.0,1.0])
a=0.5
Q=0.5
rs=1
ind1=3
ind2=3

In [16]:
ch(coord,a,Q,rs)#[ind1]

Array([[[ 0.        ,  0.26444661, -0.026329  ,  0.        ],
        [ 0.26444661,  0.        ,  0.        , -0.32223305],
        [-0.026329  ,  0.        ,  0.        ,  0.00932143],
        [ 0.        , -0.32223305,  0.00932143,  0.        ]],

       [[ 0.07114956,  0.        ,  0.        , -0.02518956],
        [ 0.        , -0.15427815, -0.04892943,  0.        ],
        [ 0.        , -0.04892943, -0.80715231,  0.        ],
        [-0.02518956,  0.        ,  0.        , -0.56260507]],

       [[-0.01133414,  0.        ,  0.        ,  0.05667071],
        [ 0.        ,  0.03914355,  0.64572185,  0.        ],
        [ 0.        ,  0.64572185, -0.04892943,  0.        ],
        [ 0.05667071,  0.        ,  0.        , -0.52800073]],

       [[ 0.        ,  0.05288932, -0.07436801,  0.        ],
        [ 0.05288932,  0.        ,  0.        ,  0.53555339],
        [-0.07436801,  0.        ,  0.        ,  0.66842162],
        [ 0.        ,  0.53555339,  0.66842162,  0.        ]]], 

In [17]:
christoffel(coord,Kerr_metric(a,Q,rs))#[ind2]

Array([[[ 0.        ,  0.26444661, -0.026329  ,  0.        ],
        [ 0.26444661,  0.        ,  0.        , -0.32223305],
        [-0.026329  ,  0.        ,  0.        ,  0.00932143],
        [ 0.        , -0.32223305,  0.00932143,  0.        ]],

       [[ 0.07114956, -0.        , -0.        , -0.02518956],
        [-0.        , -0.15427815, -0.04892943, -0.        ],
        [-0.        , -0.04892943, -0.80715231, -0.        ],
        [-0.02518956, -0.        , -0.        , -0.56260507]],

       [[-0.01133414, -0.        , -0.        ,  0.05667071],
        [-0.        ,  0.03914355,  0.64572185, -0.        ],
        [-0.        ,  0.64572185, -0.04892943, -0.        ],
        [ 0.05667071, -0.        , -0.        , -0.52800073]],

       [[-0.        ,  0.05288932, -0.07436801, -0.        ],
        [ 0.05288932, -0.        , -0.        ,  0.53555339],
        [-0.07436801, -0.        , -0.        ,  0.66842162],
        [-0.        ,  0.53555339,  0.66842162, -0.        ]]], 

In [18]:
def christoffel_from_params(coord, a, Q, rs):
    return christoffel(coord, Kerr_metric(a, Q, rs))

def compare(coord, a, Q, rs):
    c1 = christoffel_from_params(coord, a, Q, rs)
    c2 = ch(coord, a, Q, rs)
    diff = c1 - c2
    return {
        "abs_err": jnp.max(jnp.abs(diff)),
        "rel_err": jnp.max(jnp.abs(diff) / (jnp.abs(c1) + 1e-12)),
    }

In [19]:
N = 100_000

key = jax.random.PRNGKey(0)

coords = jax.random.uniform(
    key,
    shape=(N, 4),
    minval=0.5,
    maxval=2.0,
)
a_vals = jnp.linspace(0.0, 0.4, N)
Q_vals = jnp.linspace(0.0, 0.4, N)
rs_vals = jnp.ones(N)


In [20]:
coords

Array([[1.12768567, 0.82444318, 1.94798219, 1.3617508 ],
       [1.29833973, 1.03235777, 1.82451346, 1.44884059],
       [1.30070012, 0.78748082, 1.78126498, 0.58915894],
       ...,
       [0.5086475 , 1.49287072, 0.91079525, 0.82064645],
       [0.92546519, 1.99053466, 1.78960326, 0.55395039],
       [1.15906851, 0.67022824, 0.7643701 , 1.80381764]], dtype=float64)

In [21]:
v_compare = jax.vmap(compare, in_axes=(0, 0, 0, 0))

In [22]:
v_compare_jit = jax.jit(v_compare)

# First call includes compilation cost
results = v_compare_jit(coords, a_vals, Q_vals, rs_vals)


In [23]:
results

{'abs_err': Array([8.88178420e-16, 3.90798505e-14, 1.99840144e-15, ...,
        2.22044605e-16, 1.66533454e-16, 1.11022302e-15], dtype=float64),
 'rel_err': Array([7.32252642e-16, 1.15254125e-08, 3.48063357e-10, ...,
        5.81536765e-15, 1.40135142e-14, 1.25940869e-14], dtype=float64)}

In [24]:
print("Max absolute error:", jnp.max(results["abs_err"]))
print("Mean absolute error:", jnp.mean(results["abs_err"]))

print("Max relative error:", jnp.max(results["rel_err"]))
print("Mean relative error:", jnp.mean(results["rel_err"]))


Max absolute error: 3.797205863520503e-05
Mean absolute error: 3.8271026247508273e-10
Max relative error: 1.0115181841167058e-05
Mean relative error: 2.4801195675145065e-10


In [25]:
%%time 
res=v_compare_jit(coords, a_vals, Q_vals, rs_vals)
res

CPU times: user 1.54 ms, sys: 184 μs, total: 1.72 ms
Wall time: 814 μs


{'abs_err': Array([8.88178420e-16, 3.90798505e-14, 1.99840144e-15, ...,
        2.22044605e-16, 1.66533454e-16, 1.11022302e-15], dtype=float64),
 'rel_err': Array([7.32252642e-16, 1.15254125e-08, 3.48063357e-10, ...,
        5.81536765e-15, 1.40135142e-14, 1.25940869e-14], dtype=float64)}